In [169]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
# sub category parameter
# 사건사고 : 249
# 교육 : 250
# 노동 : 251
# 언론 : 254
# 환경 : 252
# 인권/복지 : 59b
# 식품/의료 : 255
# 지역 : 256
# 인물 : 276
# 사회 일반 : 257

In [251]:
def get_title(soup):
    title = soup.find(id='title_area').text
    return title

def get_content(soup):
    content = soup.find(id='newsct_article').text
    return content

def get_writer(soup):
    try:
        writer = soup.find(class_='media_end_head_journalist_name').text
    except:
        try: 
            writer = soup.find(class_='byline_p').text
        except:
            writer = 'None'
    return writer

def get_writed_at(soup):
    writed_at = soup.find(class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').text
    return writed_at

def get_press(soup):
    press = soup.find(class_='media_end_head_top_logo').find('img')['title']
    return press

In [270]:
# 사건사고

# date_list = list(map(str, range(20230201, 20230229))) + list(map(str, range(20230301, 20230320)))
# df = pd.DataFrame(columns=['title', 'writer', 'writed_at', 'content', 'press', 'main_category', 'sub_category', 'platform'])
# df.to_csv('../data/사회_사건사고.csv', index=False, encoding='utf-8-sig')

for date in date_list[1:]:
    
    BASE_URL = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=249&sid1=102&mid=shm&date={}'.format(date)
    
    title_list = []
    content_list = []
    writer_list = []
    writed_at_list = []
    press_list = []
    
    # 날짜별 전체 페이지 개수
    page = 1
    total_page = 0
    while True:
        URL = BASE_URL + '&page={}'.format(page)
        response = requests.get(URL, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        if soup.find(class_='next nclicks(fls.page)'):
            page += 10
            total_page += 10
        else:
            total_page += len(soup.find(class_='paging').find_all('a'))
            break
    
    print(total_page)

    # 날짜별 전체 기사 URL
    news_url_list = []
    for i in range(1, total_page + 1):
        URL = BASE_URL + '&page={}'.format(i)
        response = requests.get(URL, headers=headers)
        soup = BeautifulSoup(response.text,'html.parser')
        for news in soup.find(class_='content').find_all('li'):
            news_url_list.append(news.find('a')['href'])
    
    # 날짜별 기사 정보
    for url in news_url_list:
        response = requests.get(url, headers=headers)
        time.sleep(1.5)
        soup = BeautifulSoup(response.text, 'html.parser')

        title = get_title(soup)
        content = get_content(soup)
        writer = get_writer(soup)
        writed_at = get_writed_at(soup)
        press = get_press(soup)

        title_list.append(title)
        content_list.append(content)
        writer_list.append(writer)
        writed_at_list.append(writed_at)
        press_list.append(press)
    
    # 데이터프레임 저장
    df = pd.DataFrame({'title' : title_list,
                   'writer' : writer_list,
                   'writed_at' : writed_at_list,
                   'content' : content_list,
                   'press' : press_list})
    df['main_category'] = '사회'
    df['sub_category'] = '사건사고'
    df['platform'] = 'Naver'
    
    df.to_csv('../data/사회_사건사고.csv', mode='a', header=False, index=False, encoding='utf-8-sig')
    
    print('{} done'.format(date))

34
20230201 done


In [271]:
df

,title,writer,writed_at,content,press,main_category,sub_category,platform
0,종편 재승인 개입 의혹 방통위 국장 구속,공병선 기자,2023.02.01. 오후 11:50,\n\n\n\n\n\n[이미지출처=연합뉴스] 종합편성채널(종편) 재승인 과정에 개입...,아시아경제,사회,사건사고,Naver
1,'TV조선 재승인 점수조작 의혹' 방통위 국장 구속,홍인택 기자,2023.02.01. 오후 11:43,\n\n\n\n\n\n방송통신위원회 로고TV조선의 종합편성채널 재승인 심사 당시 고...,한국일보,사회,사건사고,Naver
2,"현직 경찰, 수사 정보 준 대가로 성매매업자에 금품 수수 의혹",강지수 기자,2023.02.01. 오후 11:41,\n\n\t\t\t[이데일리 강지수 기자] 현직 경찰이 성매매업자에게서 금품을 받은...,이데일리,사회,사건사고,Naver
3,"'TV조선 재승인 의혹' 방통위 국장 구속 …""증거인멸 우려""",정채영 기자,2023.02.01. 오후 11:23,\n\n\n\n\n\n종합편성채널 TV조선의 재승인 심사 과정에서 점수를 조작한 혐...,더팩트,사회,사건사고,Naver
4,'TV조선 재승인 심사과정 부당개입' 방통위 국장 구속,김정록 기자,2023.02.01. 오후 11:14,"\n\n지난달 한 차례 구속영장 기각""도망의 염려, 증거인멸의 우려"" 결국 구속\n...",노컷뉴스,사회,사건사고,Naver
...,...,...,...,...,...,...,...,...
664,수영 강습 내세워 2억여원 꿀꺽 … ‘매칭 앱’ 사기 기승,신하림 기자,2023.02.01. 오전 12:03,"\n\n춘천지법 사기 혐의 기소 30대 남성 징역형 선고전문가-수강생 연결 앱, 결...",강원일보,사회,사건사고,Naver
665,대권 꿈꾸던 안희정의 처참한 몰락[그해 오늘],한광범 기자,2023.02.01. 오전 12:03,\n\n2019년 비서 간음 혐의로 2심서 법정구속비서 방송인터뷰 직후 사과 및 지...,이데일리,사회,사건사고,Naver
666,"[결혼과 이혼] ""제발 이혼 해달라"" 불륜 남편에 요구하자 아들까지 폭행",김동현 기자,2023.02.01. 오전 12:01,\n\n\t\t\t다른 여성과 바람을 피웠음에도 오히려 뻔뻔한 태도로 부인과 아들을...,아이뉴스24,사회,사건사고,Naver
667,"[기가車] 도로 위 정체불명 낙하물, 그냥 통과하려다 대형차 뒤집혔다",김동현 기자,2023.02.01. 오전 12:01,\n\n\t\t\t고속도로에 떨어진 낙하물이 하부에 걸려 3.5t 화물차가 크게 넘...,아이뉴스24,사회,사건사고,Naver
